# <B> RAG Application

![img](https://raw.githubusercontent.com/langchain-ai/intro-to-langsmith/a61b50fc7035af9ac5958a18a9131cee84c1373b/images/simple_rag.png)

In [ ]:
import os
import tqdm as notebook_tqdm
from dotenv import load_dotenv
load_dotenv()

### Simple RAG application

In [ ]:
import ssl
import certifi
import urllib.request

ssl_context = ssl.create_default_context(cafile=certifi.where())
urllib.request.urlopen("https://docs.smith.langchain.com", context=ssl_context)

In [ ]:
import requests

url = "https://docs.smith.langchain.com"
response = requests.get(url)

print("Status Code:", response.status_code)
print("First 100 chars of content:", response.text[:100])

In [ ]:
from langsmith import traceable
from langchain_groq import ChatGroq
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

MODEL_NAME = os.getenv("GROQ_MODEL_NAME")
llm = ChatGroq(model=MODEL_NAME)
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""


nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_groq` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_groq(messages)

"""
call_groq
- Returns the chat completion output from Groq
"""
@traceable(run_type="llm")
def call_groq(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return llm.invoke(messages)

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content

In [ ]:
question = "What is LangSmith used for?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"website": "www.google.com"}})
print(ai_answer)